# Creating a New Ephys `Session` and `SessionProbe` in the Utah Organoids Pipeline


### **Overview**

This notebook guides you through the process of creating and inserting a new electrophysiology (ephys) `Session` and its associated `SessionProbe` in the Utah Organoids pipeline.
- A `Session` represents a specific time-window for ephys data analysis.
- A `SessionProbe` defines the probe used during that session, including its placement and electrodes.

By the end of this notebook, you will have:
- Selected an experiment from `Experiment` table
- Defined session and probe parameters (time window, session type, etc.)
- Created and inserted a new `Session` and `SessionProbe`


**_Note:_**

- This notebook uses example data, replace values with actual database entries.
- Session duration limits: 
    - `lfp` (Local Field Potential): <= 30 min
    - `spike_sorting`: <= 120 min


### **Key Steps**


- **Setup**

- **Step 1: Select an Experiment**

- **Step 2: Define Session Details**

- **Step 3: Insert the New `Session` and `SessionProbe`**


#### **Setup**


First, import the necessary packages for the data pipeline and essential schemas.


In [1]:
## Ensure the correct working directory

import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [2]:
import datajoint as dj
from datetime import datetime

In [3]:
from workflow.pipeline import culture, ephys, probe

[2025-03-03 11:29:57,837][INFO]: Connecting milagros@db.datajoint.com:3306
[2025-03-03 11:29:59,479][INFO]: Connected milagros@db.datajoint.com:3306


#### **Step 1: Select an Experiment**


Each `Session` must be linked to an existing **experiment** stored in the `Experiment` table. Experiments track organoid testing under various conditions. The metadata of this table was previously inserted through the Organoid Generation Pipeline using Organoids SciViz.


Check available experiments and select one to link to the new session:

In [4]:
culture.Experiment().proj(
    "experiment_end_time", "drug_name", "drug_concentration", "experiment_plan"
)

organoid_id e.g. O17,experiment_start_time,experiment_end_time,drug_name,drug_concentration concentration in uM,"experiment_plan e.g. mrna lysate, oct, protein lysate, or matrigel embedding, ephys, tracing"
E01,2024-08-30 19:43:00,2024-08-30 19:51:00,Control,nan,ephys
E02,2024-08-30 19:43:00,2024-08-30 19:51:00,Control,nan,ephys
E03,2024-08-30 19:43:00,2024-08-30 19:51:00,Control,nan,ephys
E04,2024-08-30 19:43:00,2024-08-30 19:51:00,Control,nan,ephys
E09,2024-10-30 13:09:00,2024-10-30 13:15:00,Control,nan,ephys
E10,2024-10-30 13:09:00,2024-10-30 13:15:00,Control,nan,ephys
E11,2024-10-30 13:09:00,2024-10-30 13:15:00,Control,nan,ephys
E12,2024-10-30 13:09:00,2024-10-30 13:15:00,Control,nan,ephys
MB01,2024-08-30 19:00:00,2024-08-30 19:26:00,Control,nan,ephys
MB02,2024-08-30 19:00:00,2024-08-30 19:26:00,Control,nan,ephys


Filter experiments for a specific `organoid_id`:

In [5]:
# Update the following information to match your specific needs
exp_key = dict(organoid_id="MB07")

In [6]:
(culture.Experiment & exp_key).proj(
    "experiment_end_time", "drug_name", "drug_concentration", "experiment_plan"
)

organoid_id e.g. O17,experiment_start_time,experiment_end_time,drug_name,drug_concentration concentration in uM,"experiment_plan e.g. mrna lysate, oct, protein lysate, or matrigel embedding, ephys, tracing"
MB07,2024-09-07 14:49:00,2024-09-07 15:02:00,Control,nan,ephys


Ensure that the experiment includes electrophysiology (ephys) analysis in the `experiment_plan`.

From the `Experiment` table, we can fetch the `organoid_id` and the `experiment_start_time` for the session.

In [7]:
organoid_id, experiment_start_time = (culture.Experiment & exp_key).fetch1(
    "organoid_id", "experiment_start_time"
)
organoid_id, experiment_start_time

('MB07', datetime.datetime(2024, 9, 7, 14, 49))

#### **Step 2: Define Session Details**


Once an experiment is selected, define key parameters for the session:

#### 1. Select `session_type`

Ephys analysis can be performed using either: 
- `lfp` (Local Field Potential)
- `spike_sorting`
- `both`

In [8]:
session_type = "spike_sorting"
insertion_number = 0

#### 2. Specific Time Window

Ensure the session duration fits within the experiment timeframe.

In [9]:
# Update with actual session information
start_time = "2024-09-07 14:49:00"
end_time = "2024-09-07 14:54:00"

Validate duration:

In [10]:
### The duration for `lfp` sessions should not exceed **30 minutes**, and for `spike_sorting` sessions, it should not exceed **120 minutes**. Let's check if the duration of the defined session times is within the limits.

SPIKE_SORTING_DURATION = 120  # minutes
LFP_DURATION = 30  # minutes

# Start and end time of the session. It should be within the experiment time range
start_time_str = datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
end_time_str = datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S")
duration = (end_time_str - start_time_str).total_seconds() / 60  # minutes

if session_type == "spike_sorting":
    assert (
        duration <= SPIKE_SORTING_DURATION
    ), f"Session type 'spike_sorting' duration must be less than {SPIKE_SORTING_DURATION} minutes"

elif session_type == "lfp":
    assert (
        duration <= LFP_DURATION
    ), f"Session type 'lfp' duration must be less than {LFP_DURATION} minutes"

In [11]:
new_session_key = dict(
    organoid_id=organoid_id,
    experiment_start_time=experiment_start_time,
    insertion_number=insertion_number,
    start_time=start_time,
    end_time=end_time,
)
new_session_key

{'organoid_id': 'MB07',
 'experiment_start_time': datetime.datetime(2024, 9, 7, 14, 49),
 'insertion_number': 0,
 'start_time': '2024-09-07 14:49:00',
 'end_time': '2024-09-07 14:54:00'}

In [12]:
session_info = dict(
    **new_session_key,
    session_type=session_type,
)

session_info

{'organoid_id': 'MB07',
 'experiment_start_time': datetime.datetime(2024, 9, 7, 14, 49),
 'insertion_number': 0,
 'start_time': '2024-09-07 14:49:00',
 'end_time': '2024-09-07 14:54:00',
 'session_type': 'spike_sorting'}

#### 3. Define Probe Information

In [13]:
ephys.Port()

port_id
A
B
C
D


In [14]:
probe.Probe()

probe unique identifier for this model of probe (e.g. serial number),probe_type e.g. A1x32-6mm-100-177-H32_21mm,probe_comment
12D4D,A1x32-6mm-100-177-H32_21mm,
12D51,A1x32-6mm-100-177-H32_21mm,
12D76,A1x32-6mm-100-177-H32_21mm,
12D77,A1x32-6mm-100-177-H32_21mm,
Q983,A1x32-6mm-100-177-H32_21mm,
T590,A1x32-6mm-100-177-H32_21mm,
T591,A1x32-6mm-100-177-H32_21mm,
T593,A1x32-6mm-100-177-H32_21mm,
T595,A1x32-6mm-100-177-H32_21mm,


Set up details for the probe and electrode mappings:

In [15]:
# Update the following information to match your specific needs
session_probe_info = dict(
    **new_session_key,
    probe="Q983",  # probe serial number
    port_id="C",  # Port ID ("A", "B", etc.)
    used_electrodes=[],  # electrodes used for the session; empty list if all electrodes were used
)

session_probe_info

{'organoid_id': 'MB07',
 'experiment_start_time': datetime.datetime(2024, 9, 7, 14, 49),
 'insertion_number': 0,
 'start_time': '2024-09-07 14:49:00',
 'end_time': '2024-09-07 14:54:00',
 'probe': 'Q983',
 'port_id': 'C',
 'used_electrodes': []}

With the information already defined, we can now create the new session key before inserting in the database.

#### **Step 3: Insert the New `Session` and `SessionProbe`**


Now, insert the new session and probe information into the database.

In [16]:
ephys.EphysSession.insert1(session_info, skip_duplicates=True)
ephys.EphysSessionProbe.insert1(session_probe_info, skip_duplicates=True)

Confirm session creation:

In [17]:
ephys.EphysSession & new_session_key


organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,session_type
MB07,2024-09-07 14:49:00,0,2024-09-07 14:49:00,2024-09-07 14:54:00,spike_sorting


In [18]:
ephys.EphysSessionProbe & new_session_key

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,probe unique identifier for this model of probe (e.g. serial number),port_id,"used_electrodes list of electrode IDs used in this session (if null, all electrodes are used)"
MB07,2024-09-07 14:49:00,0,2024-09-07 14:49:00,2024-09-07 14:54:00,Q983,C,=BLOB=


Check all existing `spike_sorting` sessions:

In [19]:
(ephys.EphysSession & "session_type = 'spike_sorting'")

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,session_type
E03,2024-08-30 19:43:00,0,2024-08-30 19:50:00,2024-08-30 19:55:00,spike_sorting
MB03,2024-08-30 19:00:00,0,2024-08-30 19:15:00,2024-08-30 19:20:00,spike_sorting
MB05,2024-09-07 14:49:00,0,2024-09-07 14:49:00,2024-09-07 14:54:00,spike_sorting
MB06,2024-09-07 14:49:00,0,2024-09-07 14:49:00,2024-09-07 14:54:00,spike_sorting
MB07,2024-09-07 14:49:00,0,2024-09-07 14:49:00,2024-09-07 14:54:00,spike_sorting
MB07,2024-09-07 14:49:00,0,2024-09-07 14:49:00,2024-09-07 15:02:00,spike_sorting
MB07,2024-09-07 14:49:00,0,2024-09-07 14:51:00,2024-09-07 14:52:00,spike_sorting
MB07,2024-09-07 14:49:00,0,2024-09-07 14:57:00,2024-09-07 15:00:00,spike_sorting
MB07,2024-09-07 14:49:00,0,2024-09-07 14:57:00,2024-09-07 15:02:00,spike_sorting
MB08,2024-09-07 14:49:00,0,2024-09-07 14:49:00,2024-09-07 14:54:00,spike_sorting


### **Next Steps**

Now that the session is created, you can:
- Run analysis pipelines (`RUN` notebooks)
- Explore results using `EXPLORE` notebooks.